In [15]:
import json
import os
from utils.prompt_helpers import generate_prompt_for_search
from utils.search_product_by_query_whoosh import search_index_return_json, search_index_best_effort_v2, build_merged_index
from tqdm import tqdm
from utils.openai_helpers import query_openai_model_batch_save

In [4]:

def load_user_profiles_and_interviews(
    profile_path="../Dataset/user_profiles_refined_final.json",
    interview_dir="../Results/Reference Interview"
):
    user_data = {}

    # Load the JSON user profile
    with open(profile_path, 'r') as f:
        profiles = json.load(f)

    for user_id, profile in profiles.items():
        interview_path = os.path.join(interview_dir, f"{user_id}.txt")
        try:
            with open(interview_path, 'r') as f:
                interview_text = f.read()
        except FileNotFoundError:
            interview_text = None  # or "" if you'd prefer empty string
            print(f"Warning: Interview file not found for user {user_id}")

        user_data[user_id] = {
            "profile": profile,
            "interview_text": interview_text
        }

    return user_data


In [5]:
user_data_dict = load_user_profiles_and_interviews()
user_ids = list(user_data_dict.keys())

In [6]:
user_data_dict[user_ids[0]]['profile'], user_data_dict[user_ids[0]]['interview_text']

({'user_profile': {'Gender': 'Male',
   'Age': '35-44',
   'Occupation': 'Self-employed',
   'Price Sensitivity': 'Medium',
   'Shopping Interest': 'Health and wellness, outdoor gear, home essentials, tech gadgets',
   'Brand Preference': 'Spoonk, adidas, Krinos, LifePro, Halsa, TP-Link, E EGOOZ, Hallmark, THE NORTH FACE, Thule, Spigen, VRS DESIGN, Oarkive, RYB HOME, Hanes, Amazon Basics, Apple, Garden of Life, Forge, OXO, FGO, B BANGKOK PANTS, Sanuk, KOS, Benicci',
   'Diversity Preference': 'Medium',
   'Interaction Complexity': 'Medium',
   'Tone and Style': 'Enthusiastic, supportive, practical',
   'Item Reference': 'Specific products and brands mentioned in reviews',
   'Focus Aspect': 'Quality, functionality, brand reputation'}},
 "Assistant: Hey there! I'm looking for a high-quality EMF protection sleeve for my phone. Something reliable from a reputable brand that fits easily in pockets. Any suggestions?\n\nUser: Great! To help you find the perfect EMF protection sleeve for your

In [7]:
len(user_data_dict)

120

In [8]:
user_data_dict

{'AGMQSZEQFFKH33FJQZLN7MF5QX2Q': {'profile': {'user_profile': {'Gender': 'Male',
    'Age': '35-44',
    'Occupation': 'Self-employed',
    'Price Sensitivity': 'Medium',
    'Shopping Interest': 'Health and wellness, outdoor gear, home essentials, tech gadgets',
    'Brand Preference': 'Spoonk, adidas, Krinos, LifePro, Halsa, TP-Link, E EGOOZ, Hallmark, THE NORTH FACE, Thule, Spigen, VRS DESIGN, Oarkive, RYB HOME, Hanes, Amazon Basics, Apple, Garden of Life, Forge, OXO, FGO, B BANGKOK PANTS, Sanuk, KOS, Benicci',
    'Diversity Preference': 'Medium',
    'Interaction Complexity': 'Medium',
    'Tone and Style': 'Enthusiastic, supportive, practical',
    'Item Reference': 'Specific products and brands mentioned in reviews',
    'Focus Aspect': 'Quality, functionality, brand reputation'}},
  'interview_text': "Assistant: Hey there! I'm looking for a high-quality EMF protection sleeve for my phone. Something reliable from a reputable brand that fits easily in pockets. Any suggestions?\n\

In [9]:
prompts_dict = {
    user_id: generate_prompt_for_search(user_data_dict[user_id]['profile'], user_data_dict[user_id]['interview_text'])
    for user_id in user_ids
}


In [10]:
prompt_list = list(prompts_dict.values())
len(prompt_list)

120

In [11]:
results_file_path = '../Results/generated_search_phrases/generated_search_phrases.json'
if os.path.exists(results_file_path):
    print('Results already generated, reading..')
    with open(results_file_path, 'r') as f:
        generated_search_phrases = json.load(f)

else:
    results_generated = query_openai_model_batch_save(prompt_list)
    generated_search_phrases = {}
    for i in range(len(user_ids)):
        generated_search_phrases[user_ids[i]] = json.loads(results_generated[i][0])
        
    print('saving generated search phrases to disk...')
    with open('../Results/generated_search_phrases/generated_search_phrases.json', 'w') as f:
        json.dump(generated_search_phrases, f, indent=2)
        
    

Results already generated, reading..


In [12]:
generated_search_phrases

{'AGMQSZEQFFKH33FJQZLN7MF5QX2Q': {'search_phrase': 'EMF protection sleeve minimalistic',
  'explanation': "The search phrase incorporates the user's need for a high-quality EMF protection sleeve, emphasizing the design preference for sleek, minimalistic aesthetics. It is concise, reflecting the main product category (EMF protection sleeve) and the specific style preference (minimalistic), which aligns with the user's interest in functionality and design."},
 'AFFA5IG2KKBEXHTTBUGONWM3HUVQ': {'search_phrase': 'AirPods Pro case shock-resistant colorful',
  'explanation': "The search phrase incorporates the user's need for a durable ('shock-resistant') protective case for AirPods Pro, with a preference for 'colorful' options indicating a desire for fun colors or patterns. This aligns with the conversation's emphasis on durability, shock resistance, and aesthetic variety."},
 'AEZKWI7QRZ2BOO4J5ZIRNY5YIL2Q': {'search_phrase': 'adjustable breathable posture corrector',
  'explanation': "The s

In [11]:
def batch_run_search_and_save_results(search_phrase_dict, output_dir="../Results/Step2-Retrieval_Results", top_k=100):
    os.makedirs(output_dir, exist_ok=True)
    
    fb_count = 0

    for user_id in tqdm(search_phrase_dict, desc="Processing search phrases"):
        entry = search_phrase_dict[user_id]
        search_phrase = entry.get("search_phrase", "").strip()
        
        if not search_phrase:
            print(f"Skipping {user_id} due to missing search phrase.")
            continue

        try:
            results, fb = search_index_best_effort_v2(search_phrase, top_k=top_k)
            
            if fb:
                fb_count += 1

            output_path = os.path.join(output_dir, f"{user_id}.json")
            with open(output_path, "w") as f:
                json.dump(results, f, indent=2)

            # print(f"Saved results for {user_id} -> {output_path}")
        except Exception as e:
            print(f"Error processing {user_id}: {e}")
          
    print(f"FB count: {fb_count}")  
    return

In [12]:
batch_run_search_and_save_results(generated_search_phrases)


Processing search phrases:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0/120 [00:00<?, ?it/s]

No direct matches found. Finding nearest matches...


Processing search phrases:   1%|███████▊                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 1/120 [00:10<20:04, 10.12s/it]

No direct matches found. Finding nearest matches...


Processing search phrases:   2%|███████████████████████▍                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 3/120 [00:17<09:13,  4.73s/it]

...

No direct matches found. Finding nearest matches...


Processing search phrases:  99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏       | 119/120 [09:52<00:05,  5.68s/it]

No direct matches found. Finding nearest matches...


Processing search phrases: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [09:59<00:00,  5.00s/it]

FB count: 0


In [26]:
res = search_index_best_effort('minimalist spice rack')
res

[{'asin': 'B0859ML2RG',
  'title': 'Talented Kitchen 134 Kitchen Spice Jar Labels Preprinted, Minimalist Bold All Caps White Letters and Numbers for Spice Rack, Kitchen Cabinet and Pantry Organization',
  'description': '',
  'features': 'Spice Rack Labels: Create a picture-perfect, organized kitchen or pantry with this set of 134 spice labels stickers that features 124 commonly used kitchen spices Stay Fresh: Place the seasoning labels on glass spice jars to create a functional, organized, visually appealing kitchen or pantry workspace while baking or preparing meals No Messy Handwriting: These minimalist spice label stickers feature bold white all caps font preprinted on water-resistant, clear vinyl with a self-adhesive backing that can easily be removed Uniform, Organized Kitchen: The clear spice labels will vary in size depending on the ingredient name; the letters stand 0.5-inch tall for uniform, picture-perfect spice organization Comes With: You will receive a total of 134 vinyl 

In [13]:
# Usage
json_files = [
    "/home/azureuser/cloudfiles/code/Users/preetams/Dynamic-Evaluation-of-Personaslized-Agents/Dataset/all_products_part_1.json",
    "/home/azureuser/cloudfiles/code/Users/preetams/Dynamic-Evaluation-of-Personaslized-Agents/Dataset/all_products_part_2.json"
]

In [16]:
build_merged_index(json_files)